In [25]:
from typing import List
import trendline
import crypto_data
import equities_data
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# I can add that the signal is a maximum of a fraction of atr away from the breakout line

In [10]:
def trendline_breakout_hl(data: pd.DataFrame, lookback: int, ema_period: int = 21):
    """
    Calculate trendline breakouts with specific pattern conditions:
    Long: price > EMA21 + negative resistance slope
    Short: price < EMA21 + positive resistance slope
    """
    # Calculate EMA
    data['EMA21'] = data['Close'].ewm(span=ema_period, adjust=False).mean()
    
    # Initialize arrays
    n = len(data)
    s_tl = np.full(n, np.nan)
    r_tl = np.full(n, np.nan)
    signals = np.zeros(n)
    r_slopes = np.full(n, np.nan)
    s_slopes = np.full(n, np.nan)
    
    for i in range(lookback, n):
        try:
            # Get window data
            window_close = data.iloc[i - lookback:i]['Close'].to_numpy()
            window_high = data.iloc[i - lookback:i]['High'].to_numpy()
            window_low = data.iloc[i - lookback:i]['Low'].to_numpy()
            
            # Calculate trendlines
            s_coefs, r_coefs = trendline.fit_trendlines_high_low(
                window_high, window_low, window_close
            )
            
            # Store slopes
            s_slopes[i] = s_coefs[0]
            r_slopes[i] = r_coefs[0]
            
            # Project trendlines to current bar
            s_val = s_coefs[1] + lookback * s_coefs[0]
            r_val = r_coefs[1] + lookback * r_coefs[0]
            
            s_tl[i] = s_val
            r_tl[i] = r_val
            
            # Get current price and EMA
            current_close = data.iloc[i]['Close']
            current_ema = data.iloc[i]['EMA21']

            # Average price condition
            window_average = window_close.mean()
            
            # Long condition: price > EMA + negative resistance slope
            if (window_average > current_ema) and (r_coefs[0] < 0) and (current_close > r_val):
                signals[i] = 1.0
                
            # Short condition: price < EMA + positive resistance slope    
            elif (window_average < current_ema) and (r_coefs[0] > 0) and (current_close < s_val):
                signals[i] = -1.0
                
            else:
                signals[i] = 0
                
        except Exception as e:
            signals[i] = signals[i - 1]
            continue
    
    return s_tl, r_tl, signals, data['EMA21'].to_numpy(), r_slopes, s_slopes

# Usage
# s_tl, r_tl, signals, ema, r_slopes, s_slopes = trendline_breakout_hl(data, lookback=14)

In [30]:
def plot_ohlcv(data: pd.DataFrame, lookback: int = 7):
    
    close_prices = data['Close']

    # Calculate trendlines using your original algorithm
    s_tl, r_tl, signals, ema, r_slopes, s_slopes = trendline_breakout_hl(data, lookback)
    
    # Create the main figure
    fig = go.Figure()
    
    # Add candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=np.arange(len(data)),
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close'],
            increasing_line_color='#26A69A',  # Green candles
            decreasing_line_color='#EF5350',  # Red candles
            name='Price'
        )
    )
    
    # Add support trendline
    fig.add_trace(
        go.Scatter(
            x=np.arange(len(s_tl)),
            y=s_tl,
            mode='lines',
            name='Support',
            line=dict(color='white', width=1, dash='dot'),
            showlegend=True
        )
    )
    
    # Add resistance trendline
    fig.add_trace(
        go.Scatter(
            x=np.arange(len(r_tl)),
            y=r_tl,
            mode='lines',
            name='Resistance',
            line=dict(color='white', width=1, dash='dot'),
            showlegend=True
        )
    )
    
    # Add EMA
    fig.add_trace(
        go.Scatter(
            x=np.arange(len(ema)),
            y=ema,
            mode='lines',
            name='EMA',
            showlegend=True
        )
    )
    
    # Find long and short signals directly
    long_idx = np.where(signals == 1)[0]
    long_values = close_prices[long_idx]
    
    short_idx = np.where(signals == -1)[0]
    short_values = close_prices[short_idx]
    
    # Add markers for long signals (up arrows)
    if len(long_idx) > 0:
        fig.add_trace(
            go.Scatter(
                x=long_idx,
                y=long_values,
                mode='markers',
                name='Long Signals',
                marker=dict(
                    size=12,
                    symbol='triangle-up',
                    color='lime',
                    line=dict(color='black', width=2)
                ),
                showlegend=True
            )
        )
    
    # Add markers for short signals (down arrows)
    if len(short_idx) > 0:
        fig.add_trace(
            go.Scatter(
                x=short_idx,
                y=short_values,
                mode='markers',
                name='Short Signals',
                marker=dict(
                    size=12,
                    symbol='triangle-down',
                    color='red',
                    line=dict(color='black', width=2)
                ),
                showlegend=True
            )
        )
    
    # Update layout
    fig.update_layout(
        title='Trendline Channel with Breakouts',
        xaxis_rangeslider_visible=False,
        height=800,
        yaxis=dict(
            gridcolor='rgba(128,128,128,0.1)',
            zerolinecolor='rgba(128,128,128,0.1)',
        ),
        xaxis=dict(
            gridcolor='rgba(128,128,128,0.1)',
            zerolinecolor='rgba(128,128,128,0.1)',
        ),
        showlegend=True,
        legend=dict(
            bgcolor='rgba(0,0,0,0.5)',
            bordercolor='rgba(255,255,255,0.2)'
        )
    )
    
    return fig

ticker_data = data['USB']
# Usage remains the same
fig = plot_ohlcv(ticker_data)

fig

# def plot_results(data: pd.DataFrame, signals: List[str]):


/var/folders/yz/f37kjxl93wl6pxrg2gkztj_h0000gp/T/ipykernel_10870/1075009687.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/yz/f37kjxl93wl6pxrg2gkztj_h0000gp/T/ipykernel_10870/3329526341.py:62: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/yz/f37kjxl93wl6pxrg2gkztj_h0000gp/T/ipykernel_10870/3329526341.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[p

In [23]:
# Main loop test

long_tickers = []
short_tickers = []

# data = crypto_data.download_crypto_ohlc_data()
# data = equities_data.download_equities_ohlc_data()

print(f"Analyzing {len(data.columns.levels[0])} symbols")


for ticker in data.columns.levels[0]:

    ticker_data = data[ticker].dropna(subset=['Close'])

    print(f"Analyzing {ticker}...")

    s_tl, r_tl, signals, ema, r_slopes, s_slopes = trendline_breakout_hl(ticker_data, 7)
    
    try:
        if signals[-1] == 1:
            long_tickers.append(ticker)
        elif signals[-1] == -1:
            short_tickers.append(ticker)
    except Exception as e:
        print(f'Error {e}')

print(f"\nLong tickers: {long_tickers}\nShort tickers: {short_tickers}\n")    

[*********************100%%**********************]  503 of 503 completed

3 Failed downloads:
['BRK.B']: Exception('%ticker%: No data found, symbol may be delisted')
['ETN']: ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (period=1y)')


Data downloaded successfully.
Analyzing 503 symbols
Analyzing A...
Analyzing AAPL...
Analyzing ABBV...
Analyzing ABNB...
Analyzing ABT...
Analyzing ACGL...
Analyzing ACN...
Analyzing ADBE...
Analyzing ADI...
Analyzing ADM...
Analyzing ADP...
Analyzing ADSK...
Analyzing AEE...
Analyzing AEP...
Analyzing AES...
Analyzing AFL...
Analyzing AIG...
Analyzing AIZ...
Analyzing AJG...
Analyzing AKAM...
Analyzing ALB...
Analyzing ALGN...
Analyzing ALL...
Analyzing ALLE...
Analyzing AMAT...
Analyzing AMCR...
Analyzing AMD...
Analyzing AME...
Analyzing AMGN...
Analyzing AMP...
Analyzing AMT...
Analyzing AMZN...
Analyzing ANET...
Analyzing ANSS...
Analyzing AON...
Analyzing AOS...
Analyzing APA...
Analyzing APD...
Analyzing APH...
Analyzing APO...
Analyzing APTV...
Analyzing ARE...
Analyzing ATO...
Analyzing AVB...
Analyzing AVGO...
Analyzing AVY...
Analyzing AWK...
Analyzing AXON...
Analyzing AXP...
Analyzing AZO...
Analyzing BA...
Analyzing BAC...
Analyzing BALL...
Analyzing BAX...
Analyzing BBY.